In [9]:
import os
import cv2
import numpy as np
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt

In [10]:
# Định nghĩa TARGET_SIZE
TARGET_SIZE = (256, 256)

In [11]:
def crop_image(image, threshold=10):
    """
    Cắt ảnh dermoscopic để loại bỏ vùng đen.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return image
    contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(contour)
    return image[y:y+h, x:x+w]

In [12]:
def shades_of_gray(image, p=6):
    """
    Áp dụng thuật toán Shades of Gray để chuẩn hóa màu sắc.
    """
    if image.shape[2] != 3:
        raise ValueError("Ảnh đầu vào phải là ảnh RGB (3 kênh).")
    img_float = image.astype(np.float32)
    norm = np.power(np.sum(np.power(img_float, p), axis=2), 1.0 / p)
    norm = np.expand_dims(norm, axis=2)
    img_normalized = img_float / (norm + 1e-6)
    img_normalized = np.clip(img_normalized * 255.0, 0, 255).astype(np.uint8)
    return img_normalized

In [ ]:
def preprocess_image(image_path, target_size=TARGET_SIZE, model_type='B0'):
    """
    Đọc và tiền xử lý ảnh.
    """
    size_map = {'B0': (224, 224), 'B1': (240, 240), 'B2': (260, 260), 'B3': (300, 300),
                'B4': (380, 380), 'B5': (456, 456), 'B6': (528, 528)}
    target_size = size_map.get(model_type, target_size)

    try:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Không thể đọc ảnh: {image_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = crop_image(image)
        image = shades_of_gray(image, p=6)
        h, w = image.shape[:2]
        scale = min(target_size[0] / h, target_size[1] / w)
        new_h, new_w = int(h * scale), int(w * scale)
        image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
        padded_image = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)
        pad_h = (target_size[0] - new_h) // 2
        pad_w = (target_size[1] - new_w) // 2
        padded_image[pad_h:pad_h + new_h, pad_w:pad_w + new_w] = image
        return padded_image
    except Exception as e:
        print(f"Lỗi khi xử lý ảnh {image_path}: {e}")
        return None

In [14]:
def get_augmentation_pipeline(save_mode=False):
    """
    Tạo pipeline tăng cường dữ liệu sử dụng Albumentations.
    """
    transforms = [
        A.SmallestMaxSize(max_size=TARGET_SIZE[0] * 2, p=1.0),  # Resize cạnh ngắn nhất
        A.RandomCrop(height=TARGET_SIZE[0], width=TARGET_SIZE[1], p=1.0),  # Cắt ngẫu nhiên
        A.RandomRotate90(p=0.5),  # Xoay 90°, 180°, 270°
        A.HorizontalFlip(p=0.5),  # Lật ngang
        A.VerticalFlip(p=0.5),  # Lật dọc
        A.Affine(
            scale=(0.8, 1.2),  # Phóng to/thu nhỏ
            translate_percent=(-0.1, 0.1),  # Dịch chuyển
            rotate=(-45, 45),  # Xoay
            shear=(-10, 10),  # Shear
            p=0.5
        ),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.GaussNoise(
            std_range=(10.0/255, 50.0/255),  # Chuyển var_limit sang std_range
            mean_range=(0.0, 0.0),
            per_channel=True,
            p=0.5
        ),
        A.CoarseDropout(
            num_holes_range=(1, 1),  # 1 lỗ, khớp với ISIC 2019
            hole_height_range=(16, 16),  # Kích thước lỗ 16 pixel
            hole_width_range=(16, 16),  # Kích thước lỗ 16 pixel
            fill=0,  # Điền màu đen
            p=0.3
        ),
    ]
    if not save_mode:
        transforms.append(A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)))
        transforms.append(A.ToTensorV2())
    return A.Compose(transforms)

In [ ]:
def prepare_dataset(data_dir, output_dir, target_size=TARGET_SIZE, model_type='B0'):
    """
    Xử lý và lưu trữ dữ liệu đã tiền xử lý và tăng cường.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    augmentation_pipeline = get_augmentation_pipeline(save_mode=True)

    if os.path.isfile(data_dir) and data_dir.lower().endswith(('.jpg', '.png')):
        filename = os.path.basename(data_dir)
        processed_image = preprocess_image(data_dir, target_size, model_type)
        if processed_image is None:
            return
        output_path = os.path.join(output_dir, f"processed_{filename}")
        Image.fromarray(processed_image).save(output_path)
        for i in range(3):
            augmented = augmentation_pipeline(image=processed_image)
            aug_image = augmented['image']
            aug_output_path = os.path.join(output_dir, f"aug_{i}_{filename}")
            Image.fromarray(aug_image).save(aug_output_path)
    else:
        if not os.path.exists(data_dir):
            raise ValueError(f"Thư mục không tồn tại: {data_dir}")
        for filename in os.listdir(data_dir):
            if filename.lower().endswith(('.jpg', '.png')):
                image_path = os.path.join(data_dir, filename)
                processed_image = preprocess_image(image_path, target_size, model_type)
                if processed_image is None:
                    continue
                output_path = os.path.join(output_dir, f"processed_{filename}")
                Image.fromarray(processed_image).save(output_path)
                for i in range(3):
                    augmented = augmentation_pipeline(image=processed_image)
                    aug_image = augmented['image']
                    aug_output_path = os.path.join(output_dir, f"aug_{i}_{filename}")
                    Image.fromarray(aug_image).save(aug_output_path)

In [ ]:
def display_sample_images(data_dir, output_dir, num_samples=3):
    """
    Hiển thị ảnh gốc, ảnh tiền xử lý và ảnh tăng cường.
    """
    augmentation_pipeline = get_augmentation_pipeline(save_mode=True)
    plt.figure(figsize=(12, 4 * num_samples))
    for i, filename in enumerate(os.listdir(data_dir)[:num_samples]):
        if filename.lower().endswith(('.jpg', '.png')):
            image_path = os.path.join(data_dir, filename)
            processed_image = preprocess_image(image_path)
            if processed_image is None:
                continue
            augmented = augmentation_pipeline(image=processed_image)
            aug_image = augmented['image']

            plt.subplot(num_samples, 3, i*3 + 1)
            plt.title("Ảnh gốc")
            plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.subplot(num_samples, 3, i*3 + 2)
            plt.title("Ảnh đã tiền xử lý")
            plt.imshow(processed_image)
            plt.axis('off')
            plt.subplot(num_samples, 3, i*3 + 3)
            plt.title("Ảnh đã tăng cường")
            plt.imshow(aug_image)
            plt.axis('off')
    plt.tight_layout()
    plt.show()

: 

In [ ]:
# Ví dụ sử dụng
data_dir = "F:/CV final/ISIC_2019_Training_Input"
output_dir = "F:/CV final/processed_train_isic2019"
prepare_dataset(data_dir, output_dir, model_type='B0')
display_sample_images(data_dir, output_dir, num_samples=3)